In [1]:
# import pymongo

# Connect to MongoDB
# client = pymongo.MongoClient('localhost', 27017)

# Access the 'dd-usda' database and 'publications' collection
# db = client['dd-usda']
# collection = db['publications']


In [ ]:
# query = {
#     'author_american': True,
#     '$or': [
#         {
#             'aliases': {
#                 '$in': [
#                     "USDA", "USDA Census", "US Department of Agriculture",
#                     "United States Department of Agriculture", "NASS", "NASS Census of Agriculture"
#                 ]
#             }
#         },
#         {
#             'second_alias': {
#                 '$in': [
#                     "USDA", "USDA Census", "US Department of Agriculture",
#                     "United States Department of Agriculture", "NASS", "NASS Census of Agriculture"
#                 ]
#             }
#         }
#     ]
# }
# # Define the projection to include only 'institutions' and 'year'
# projection = {
#     'institutions': 1,
#     'year': 1,
#     '_id': 0  # Exclude the '_id' field
# }

# # Retrieve all documents with the specified fields
# cursor = collection.find(query, projection)
# documents = list(cursor)


In [4]:
# # Initialize an empty list to store institution-year pairs
# institution_year_list = []

# for doc in documents:
#     year = doc.get('year')
#     institutions = doc.get('institutions', [])
    
#     # Skip documents without a year
#     if year is None:
#         continue
    
#     # Process each institution
#     for institution in institutions:
#         name = institution.get('name')
#         if name:
#             # Append the institution and year to the list
#             institution_year_list.append({'year': int(year), 'name': name.strip()})


In [ ]:
# import pandas as pd

# # Create a DataFrame from the institution-year list
# openalex_institutions = pd.DataFrame(institution_year_list)

# # Remove duplicates
# openalex_institutions.drop_duplicates(inplace=True)

# # Ensure 'year' is of integer type
# openalex_institutions['year'] = openalex_institutions['year'].astype(int)

# # Display the first few rows
# print("OpenAlex Institutions DataFrame:")
# print(openalex_institutions.head())


In [1]:
import pandas as pd

# Create a DataFrame from the institution-year list
openalex_institutions = pd.read_csv("openalex_institutions.csv")

# Display the first few rows
print("OpenAlex Institutions DataFrame:")
print(openalex_institutions.head())


OpenAlex Institutions DataFrame:
   Unnamed: 0  year                                      name
0           0  2017             Union of Concerned Scientists
1           1  2017        Pacific Northwest Research Station
2           2  2017                         US Forest Service
3           3  2017                     Iowa State University
4           4  2017  National Agricultural Statistics Service


C:\Users\rafal\AppData\Local\Temp\ipykernel_11228\2728894023.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# openalex_institutions.to_csv("openalex_institutions.csv")

In [3]:
# Load the CSV file
# From Sciserver, database DemocratizingData_USDA_2023, query: select distinct publication.year, publication_affiliation.institution_name from publication join publication_affiliation on publication.id=publication_affiliation.publication_id
scopus_institutions = pd.read_csv('scopus_institutuions.csv')

# Remove duplicates
scopus_institutions.drop_duplicates(inplace=True)

# Ensure 'year' is of integer type
scopus_institutions['year'] = scopus_institutions['year'].astype(int)

# Strip whitespace from names
scopus_institutions['name'] = scopus_institutions['name'].str.strip()

# Display the first few rows
print("\nScopus Institutions DataFrame:")
print(scopus_institutions.head())



Scopus Institutions DataFrame:
   year                               name
0  1995  Center for Mental Health Services
1  1996          Southern Research Station
2  1999         Erie County Medical Center
3  1999     U.S. Department of Agriculture
4  2000                                NaN


In [4]:
# Combine years from both datasets
years = sorted(set(openalex_institutions['year']).union(set(scopus_institutions['year'])))


In [5]:
# Initialize a list to store the results
results = []

for year in years:
    # Get institutions for the current year from OpenAlex
    oa_institutions_set = set(
        openalex_institutions[openalex_institutions['year'] == year]['name'].str.lower().str.strip()
    )
    
    # Get institutions for the current year from Scopus
    scopus_institutions_set = set(
        scopus_institutions[scopus_institutions['year'] == year]['name'].str.lower().str.strip()
    )
    
    # Institutions only in OpenAlex
    only_openalex = oa_institutions_set - scopus_institutions_set
    
    # Institutions only in Scopus
    only_scopus = scopus_institutions_set - oa_institutions_set
    
    # Institutions in both
    both = oa_institutions_set & scopus_institutions_set
    
    # Append the counts to the results list
    results.append({
        'year': year,
        'only_openalex': len(only_openalex),
        'only_scopus': len(only_scopus),
        'both': len(both)
    })


In [6]:
# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Sort the DataFrame by year
results_df = results_df.sort_values('year')

# Display the results
print("\nInstitution Counts by Year and Source:")
print(results_df)



Institution Counts by Year and Source:
    year  only_openalex  only_scopus  both
0   1991              0            1     0
1   1992              0            1     0
2   1993              0            5     0
3   1994              0           13     0
4   1995              0            5     0
5   1996              0           20     0
6   1997              0           23     0
7   1998              0           21     0
8   1999              0           26     0
9   2000              0           18     0
10  2001              0           21     0
11  2002              0           29     0
12  2003              0           44     0
13  2004              0           60     0
14  2005              0           74     0
15  2006              0           98     0
16  2007              0           56     0
17  2008              0           89     0
18  2009              0          127     0
19  2010              0          108     0
20  2011              0          100     0
21  2012      